# Практическое задание №1 по теме "Предобработка текста".

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

Задания:

1) Заменим html-сущности (к примеру: &lt; &gt; &amp;). "&lt;" заменим на “<” и "&amp;" заменим на “&”)""". Сделаем это с помощью HTMLParser.unescape(). Всю предобработку делаем в новом столбце 'clean_tweet'

2) Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function).

3) Изменим регистр твитов на нижний с помощью .lower().

4) Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

5) Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

6) Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

7) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'.

8) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.

9) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'.

10) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1]).

11) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

12) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

13) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

14) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

15) Сохраним результат предобработки в pickle-файл.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}



short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}


emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedication #willpower to find #newmaterialsâ¦
1,31964,@user #white #supremacists want everyone to see the new â #birdsâ #movie â and hereâs why
2,31965,safe ways to heal your #acne!! #altwaystoheal #healthy #healing!!
3,31966,"is the hp and the cursed child book up for reservations already? if yes, where? if no, when? ððð #harrypotter #pottermore #favorite"
4,31967,"3rd #bihday to my amazing, hilarious #nephew eli ahmir! uncle dave loves you and missesâ¦"


In [6]:
combine_df = train_df.append(test_df, ignore_index = True, sort = False)
combine_df.head()

<ipython-input-6-c272e81ebc56>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_df = train_df.append(test_df, ignore_index = True, sort = False)


,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


In [7]:
print(combine_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      49159 non-null  int64  
 1   label   31962 non-null  float64
 2   tweet   49159 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB
None


2. Удалим @user из всех твитов с помощью паттерна "@[\w]*". Для этого создадим функцию: 
 - для того, чтобы найти все вхождения паттерна в тексте, необходимо использовать re.findall(pattern, input_txt)
 - для для замены @user на пробел, необходимо использовать re.sub()
при применении функции необходимо использовать np.vectorize(function)


### Все замечательно работает и без re.findall. Его прикрутить не смог. Ошибка.

In [8]:
def user(text):
  try:
    text = re.sub(r'@[\w]*', '', text)
    return text
  except:
    return text

combine_df['tweet'] = combine_df['tweet'].apply(user)

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


3. Изменим регистр твитов на нижний с помощью .lower()

In [9]:
def low(text):
  return text.lower()

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: low(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


4. Заменим сокращения с апострофами (пример: ain't, can't) на пробел, используя apostrophe_dict. Для этого необходимо сделать функцию: для каждого слова в тексте проверить (for word in text.split()), если слово есть в словаре apostrophe_dict в качестве ключа (сокращенного слова), то заменить ключ на значение (полную версию слова).

In [10]:
def apostrophe(text):
  for word in text.split():
    if word in apostrophe_dict.keys():
      word_2 = apostrophe_dict[word]
      text = re.sub(word, word_2, text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: apostrophe(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


5. Заменим сокращения на их полные формы, используя short_word_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

In [11]:
def short(text):
  for word in text.split():
    if word in short_word_dict.keys():
      word_2 = short_word_dict[word]
      text = re.sub(word, word_2, text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: short(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love yoyou take with yoyou all the time in yoyourð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


6. Заменим эмотиконы (пример: ":)" = "happy") на пробелы, используя emoticon_dict. Для этого воспользуемся функцией, используемой в предыдущем пункте.

### Со скобками проблема. Появляется ошибка со скобкой. Работает только так:

In [12]:
def smile(text):
  for word in text.split():
    if word in emoticon_dict.keys():
      word_2 = emoticon_dict[word]
      try:
        text = re.sub(word, word_2, text)
      except:
        if word == ':(':
          text = re.sub(r'[:(]', word_2, text)
        elif word == ':)':
          text = re.sub(r'[:)]', word_2, text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: smile(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0.0,thanks for #lyft credit i cannot use cause they do not offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0.0,bihday your majesty
3,4,0.0,#model i love yoyou take with yoyou all the time in yoyourð±!!! ðððð ð¦ð¦ð¦
4,5,0.0,factsguide: society now #motivation


7. Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [13]:
def punkt(text):
  for word in text.split():
    text = re.sub(r'[^\w\s]', '', text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: punkt(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love yoyou take with yoyou all the time in yoyourð ðððð ððð
4,5,0.0,factsguide society now motivation


8. Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [14]:
def spc(text):
  for word in text.split():
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: spc(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love yoyou take with yoyou all the time in yoyour
4,5,0.0,factsguide society now motivation


9. Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [15]:
def num(text):
  for word in text.split():
    text = re.sub(r'[^a-zA-Z]', ' ', text)
  return text

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: num(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit i cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model i love yoyou take with yoyou all the time in yoyour
4,5,0.0,factsguide society now motivation


10. Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [16]:
def ch1(text):
  text_new = ' '.join([w for w in text.split() if len(w)>1])
  return text_new

combine_df['tweet'] = combine_df['tweet'].apply(lambda x: ch1(x))

combine_df.head()

,id,label,tweet
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked
2,3,0.0,bihday your majesty
3,4,0.0,model love yoyou take with yoyou all the time in yoyour
4,5,0.0,factsguide society now motivation


11. Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [17]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
nltk.download('punkt')

tweet_token = []

for i in combine_df['tweet']:
  x = tknz.word_tokenize(i)
  tweet_token.append(x)

combine_df['tweet_token'] = tweet_token
combine_df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,label,tweet,tweet_token
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]"
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]"
3,4,0.0,model love yoyou take with yoyou all the time in yoyour,"[model, love, yoyou, take, with, yoyou, all, the, time, in, yoyour]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]"


12. Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))

stop_token = []

for i in combine_df['tweet_token']:
  x = [word for word in i if not word in stop_words]
  stop_token.append(x)

combine_df['tweet_token_filtered'] = stop_token
combine_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,label,tweet,tweet_token,tweet_token_filtered
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]"
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]"
3,4,0.0,model love yoyou take with yoyou all the time in yoyour,"[model, love, yoyou, take, with, yoyou, all, the, time, in, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]"


13. Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [19]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

stemmed_token = []
stemmed_token_snt = []

for i in combine_df['tweet_token_filtered']:
  for j in i:
    x = stemmer.stem(j)
    stemmed_token_snt.append(x)
  stemmed_token.append(stemmed_token_snt)
  stemmed_token_snt = []

combine_df['tweet_stemmed'] = stemmed_token
combine_df.head()

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]"
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]"
3,4,0.0,model love yoyou take with yoyou all the time in yoyour,"[model, love, yoyou, take, with, yoyou, all, the, time, in, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]"


14. Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [20]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
pos = wordnet.VERB

lemm_token = []
lemm_token_snt = []

for i in combine_df['tweet_token_filtered']:
  for j in i:
    x = lemmatizer.lemmatize(j, pos)
    lemm_token_snt.append(x)
  lemm_token.append(lemm_token_snt)
  lemm_token_snt = []

combine_df['tweet_lemmatized'] = lemm_token
combine_df.sample(5)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
47864,47865,NaN,heal with the lawofattraction loa altwaystoheal healthy wellness,"[heal, with, the, lawofattraction, loa, altwaystoheal, healthy, wellness]","[heal, lawofattraction, loa, altwaystoheal, healthy, wellness]","[heal, lawofattract, loa, altwaystoh, healthi, well]","[heal, lawofattraction, loa, altwaystoheal, healthy, wellness]"
36548,36549,NaN,sioux bull up you will dominate your bull and you will direct it whatever you want it to do when you st,"[sioux, bull, up, you, will, dominate, your, bull, and, you, will, direct, it, whatever, you, want, it, to, do, when, you, st]","[sioux, bull, dominate, bull, direct, whatever, want, st]","[sioux, bull, domin, bull, direct, whatev, want, st]","[sioux, bull, dominate, bull, direct, whatever, want, st]"
1819,1820,0.0,we welcome all our new stylemile members to shopping with your saavoyagermiles,"[we, welcome, all, our, new, stylemile, members, to, shopping, with, your, saavoyagermiles]","[welcome, new, stylemile, members, shopping, saavoyagermiles]","[welcom, new, stylemil, member, shop, saavoyagermil]","[welcome, new, stylemile, members, shop, saavoyagermiles]"
28639,28640,0.0,happy fathers day familyfriends love,"[happy, fathers, day, familyfriends, love]","[happy, fathers, day, familyfriends, love]","[happi, father, day, familyfriend, love]","[happy, father, day, familyfriends, love]"
2558,2559,0.0,alright it has it is your turn tomorrow let us do this compete goblue,"[alright, it, has, it, is, your, turn, tomorrow, let, us, do, this, compete, goblue]","[alright, turn, tomorrow, let, us, compete, goblue]","[alright, turn, tomorrow, let, us, compet, goblu]","[alright, turn, tomorrow, let, us, compete, goblue]"


15. Сохраним результат предобработки в pickle-файл.

In [21]:
import pickle

with open('/content/drive/MyDrive/data.pickle', 'wb') as f:
  pickle.dump(combine_df, f)

with open('/content/drive/MyDrive/data.pickle', 'rb') as f:
  data_new = pickle.load(f)

data_new

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.0,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love yoyou take with yoyou all the time in yoyour,"[model, love, yoyou, take, with, yoyou, all, the, time, in, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"
...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory leftright polarisation trump uselections leadership politics brexit blm gt,"[thought, factory, leftright, polarisation, trump, uselections, leadership, politics, brexit, blm, gt]","[thought, factory, leftright, polarisation, trump, uselections, leadership, politics, brexit, blm, gt]","[thought, factori, leftright, polaris, trump, uselect, leadership, polit, brexit, blm, gt]","[think, factory, leftright, polarisation, trump, uselections, leadership, politics, brexit, blm, gt]"
49155,49156,NaN,feeling like mermaid hairflip neverready formal wedding gown dresses mermaid,"[feeling, like, mermaid, hairflip, neverready, formal, wedding, gown, dresses, mermaid]","[feeling, like, mermaid, hairflip, neverready, formal, wedding, gown, dresses, mermaid]","[feel, like, mermaid, hairflip, neverreadi, formal, wed, gown, dress, mermaid]","[feel, like, mermaid, hairflip, neverready, formal, wed, gown, dress, mermaid]"
49156,49157,NaN,hillary campaigned today in ohioomg amp used words like assetsampliability never once did clinton say theeword radicalization,"[hillary, campaigned, today, in, ohioomg, amp, used, words, like, assetsampliability, never, once, did, clinton, say, theeword, radicalization]","[hillary, campaigned, today, ohioomg, amp, used, words, like, assetsampliability, never, clinton, say, theeword, radicalization]","[hillari, campaign, today, ohioomg, amp, use, word, like, assetsampli, never, clinton, say, theeword, radic]","[hillary, campaign, today, ohioomg, amp, use, word, like, assetsampliability, never, clinton, say, theeword, radicalization]"
49157,49158,NaN,happy at work conference right mindset leads to cultureofdevelopment organizations work mindset,"[happy, at, work, conference, right, mindset, leads, to, cultureofdevelopment, organizations, work, mindset]","[happy, work, conference, right, mindset, leads, cultureofdevelopment, organizations, work, mindset]","[happi, work, confer, right, mindset, lead, cultureofdevelop, organ, work, mindset]","[happy, work, conference, right, mindset, lead, cultureofdevelopment, organizations, work, mindset]"
